# Calendar generator

This script generates a calendar to be imported primarily to Google Calendar. Main features:

- List the matches of the selected competition and team
- Generates an output
- played matches has a score. It is also displayed.

In [17]:
#import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import arrow
from datetime import datetime
from ics import Calendar, Event

# Setting up parameters

In [9]:
#match_id = '76648'
#match_id = '75806'
#match_url = 'http://www.waterpolo.hu/meccs/' + match_id

competition_id = '680' # Fiú országos bajnokság
#competition_id = '704' # Gyermek I. Budapest bajnokság
base_url = 'https://waterpolo.hu'
competition_url = 'https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=' + competition_id
selected_team = "KSI"

# Parsing competition using BeutifulSoup

In [10]:
page = requests.get(competition_url, timeout=10)
#soup = BeautifulSoup(page.content, 'html-parser')
soup = BeautifulSoup(page.content, 'html5lib')
matches = soup.find_all("tr", class_='gamerow')


In [11]:
competition = list()

for match in matches:
    mtch = {'teams': []}
    competition.append(mtch)
    mtch['rawdata'] = mtch
    location = match.td
    for i, row in enumerate(match.find_all('td')):
        if row.text.startswith("202"):
            mtch['date_str'] = row.text.strip()
        elif i == 6 and row.text.strip() != '':
            mtch['location'] = row.text.strip()
    for link in match.find_all("a"):
        href = link['href']
        if '/csapat/'  in href:
            mtch['teams'].append(link.text)
        elif '/MatchReturnData' in href:
            mtch['report_url'] = href
        elif '/meccs/' in href:
            mtch['match_url'] = href
            mtch['result'] = link.text
        elif 'www.google.com/calendar/event' in href:
            mtch['google_calendar'] = href
        else:
            print(' nem feldolgozott sor: ', link)
random.sample(competition, 5)

[{'teams': ['AQUA SE', 'ASI DSE'],
  'rawdata': {...},
  'date_str': '2022. nov. 13. 12:00',
  'location': 'Nyíregyháza',
  'google_calendar': 'http://www.google.com/calendar/event?action=TEMPLATE&text=Fi%C3%BA+Gyermek+Orsz%C3%A1gos+Bajnoks%C3%A1g%3A+AQUA+SE+-+ASI+DSE&dates=20221113T120000/20221113T133000&sprop=Weboldal:http://waterpolo.hu&details=Fi%C3%BA+Gyermek+Orsz%C3%A1gos+V%C3%ADzilabda+Bajnoks%C3%A1g%0AAlapszakasz+-+A2+-+Keleti+csoport++-+8.+fordul%C3%B3++-+2022-11-13+12%3A00%0AAQUA+SE+-+ASI+DSE+%28Ny%C3%ADregyh%C3%A1za%29%0AJ%C3%A1t%C3%A9kvezet%C5%91k%3A+S%C3%A1ndor+P%C3%A9ter%2C+Solt%C3%A9sz+Norbert+Andr%C3%A1s&location=Ny%C3%ADregyh%C3%A1za&trp=true',
  'match_url': '/meccs/76281',
  'result': '5-14',
  'report_url': '/store_pdf/MatchReturnData_680_76281.pdf'},
 {'teams': [], 'rawdata': {...}},
 {'teams': ['Vasas SC', 'KESI'],
  'rawdata': {...},
  'date_str': '2022. okt. 27. 12:15',
  'location': 'Komjádi',
  'google_calendar': 'http://www.google.com/calendar/event?action=TE

## Cleaning data

In [12]:
for match in competition:
    if len(match) < 5:
        del match

def filter_competition_by_team(competition: list, team: str) -> list:
    filtered = list()
    for match in competition:
        if len(match) > 3:
            for t in match['teams']:
                if team in t:
                    filtered.append(match)
                    break
    return filtered

competition_ksi = filter_competition_by_team(competition, "KSI")

for i, match in enumerate(competition_ksi):
    print(i, match['date_str'], match['teams'], match['result'] if match['result'] != '0-0' else '')


0 2022. szept. 25. 11:15 ['KSI SE', 'KESI'] 18-6
1 2022. okt. 02. 12:15 ['Blamax Kaposvári VK', 'KSI SE'] 1-26
2 2022. okt. 09. 15:15 ['KSI SE', 'Legrand Szentes'] 21-4
3 2022. okt. 12. 17:00 ['Honvéd', 'KSI SE'] 2-23
4 2022. okt. 16. 11:30 ['AVUS', 'KSI SE'] 1-26
5 2022. okt. 30. 12:15 ['KSI SE', 'CPVSE'] 23-1
6 2022. nov. 06. 15:15 ['KSI SE', 'Szolnoki Dózsa'] 7-6
7 2022. nov. 13. 10:00 ['Genesys OSC Újbuda', 'KSI SE'] 7-19
8 2022. nov. 20. 12:15 ['KSI SE', 'Pécsi Sportiskola'] 9-8
9 2022. nov. 27. 12:15 ['Tigra-Zf-Eger', 'KSI SE'] 
10 2022. dec. 04. 11:15 ['KSI SE', 'Vasas SC'] 20-7
11 2022. dec. 11. 12:30 ['FTC-Telekom', 'KSI SE'] 7-11
12 2022. dec. 18. 11:15 ['KSI SE', 'UVSE I.'] 7-13
13 2023. febr. 05. 10:00 ['SZVPS', 'KSI SE'] 
14 2023. febr. 12. 11:15 ['KSI SE', 'BVSC-Zugló'] 


## Generating calendar

In [18]:
calendar = Calendar()

for match in competition_ksi:
    event = Event()
    result = (': ' + match['result']) if match['result'] != '0-0' else ''
    event.name = f'{match["teams"][0]} - {match["teams"][1]}{result}'
    event.description = f'<a href="{base_url}{match["match_url"]}">Adatlap</a>\n<a href="{competition_url}">Bajnokság</a>'
    event.begin = arrow.get(match['date_str'], "YYYY. MMM. D. H:mm", locale="hu", tzinfo="CET")
    event.duration = {'hours': 1}
    event.last_modified = datetime.now()
    event.location = match['location']
    calendar.events.add(event)


print(calendar.serialize())

BEGIN:VCALENDAR
VERSION:2.0
PRODID:ics.py - http://git.io/lLljaA
BEGIN:VEVENT
DESCRIPTION:<a href="https://waterpolo.hu/meccs/76650">Adatlap</a>\n<a href="https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=680">Bajnokság</a>
DURATION:PT1H
LAST-MODIFIED:20230109T233438Z
LOCATION:Komjádi
DTSTART:20221009T131500Z
SUMMARY:KSI SE - Legrand Szentes: 21-4
UID:d575f611-36a7-4f9b-97f8-58deb3da8940@d575.org
END:VEVENT
BEGIN:VEVENT
DESCRIPTION:<a href="https://waterpolo.hu/meccs/76682">Adatlap</a>\n<a href="https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=680">Bajnokság</a>
DURATION:PT1H
LAST-MODIFIED:20230109T233438Z
LOCATION:Széchy T. u. Margitsz.
DTSTART:20221106T141500Z
SUMMARY:KSI SE - Szolnoki Dózsa: 7-6
UID:33edb556-5d46-45bd-865a-c9ab448ed30f@33ed.org
END:VEVENT
BEGIN:VEVENT
DESCRIPTION:<a href="https://waterpolo.hu/meccs/76694">Adatlap</a>\n<a href="https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=680">Bajnokság</a>
DURATION:PT1H
LAST-MODIFIED:20230109T233438Z
LOCATION:Ny